## Initial setup

**folders need to be created before running the notebook: best_classify_models_main_result and classify_scores_main_result**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
print(tf.__version__)
import torch
print(torch.__version__)
import matplotlib
print(matplotlib.__version__)

2.8.0
1.10.0+cu111
3.2.2


In [ ]:
!nvidia-smi

Fri Jan 21 19:41:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Other imports
! pip install tensorflow_addons
! pip install tensorflow_io

import os
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
from imutils import paths
from tqdm import tqdm
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import tensorflow_io as tfio
import tensorflow_hub as hub
import numpy as np
import cv2
import pandas as pd
import seaborn as sns
from scipy.stats import mannwhitneyu
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import sklearn.manifold
from sklearn.metrics.pairwise import cosine_similarity as cos
from sympy.utilities.iterables import multiset_permutations
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler
from IPython.display import Image, display


import zipfile
import concurrent.futures

# Random seed fix
random_seed = 42
tf.random.set_seed(random_seed)
np.random.seed(random_seed)

     |████████████████████████████████| 1.1 MB 6.4 MB/s 
     |████████████████████████████████| 23.4 MB 1.8 MB/s 


## Dataset gathering and preparation

In [ ]:
training_batch_size = 4

BATCH_SIZE = training_batch_size

imageSize = 224

category_names = ['bundle', 'dispersed', 'network', 'singular']
color_method = ['C0', 'C1', 'C2', 'C3', 'C4']
color = ['black', 'magenta', 'cyan', 'yellow']
marker = ['o', 's', '<', '>', '^']
seaborn_palette = sns.color_palette("colorblind")

In [ ]:
np.random.seed(random_seed)
peptide_morph_train_path = "/content/drive/MyDrive/TEM image datasets/2022-nanowire-morphology"
peptide_morph_images_train = list(paths.list_files(basePath=peptide_morph_train_path, validExts='jpg'))
peptide_morph_images_train = np.random.choice(np.array(peptide_morph_images_train), len(peptide_morph_images_train), replace=False)
print(len(peptide_morph_images_train))

400


In [ ]:
train_labels = []
for i in range(peptide_morph_images_train.shape[0]):
  train_label = peptide_morph_images_train[i].split("/")[-2]
  train_labels.append(train_label)
le = LabelEncoder()
peptide_morph_train_enc = le.fit_transform(train_labels)

In [ ]:
# Image preprocessing utils
@tf.function
def parse_images(image_path):
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    # image = tfio.experimental.image.decode_tiff(image_string)[:, :, :3]   # in the doc, it transforms tiff to 4 channels, with additional channel of opacity which is not needed.
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[imageSize, imageSize])

    return image

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(peptide_morph_images_train)
train_ds = (
    train_ds
    .map(parse_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # .shuffle(200)
    .batch(training_batch_size
          #  , drop_remainder=True
           )
    .prefetch(tf.data.experimental.AUTOTUNE)
)

datagen = tf. keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

## Initiate self-supervised models

In [ ]:
Resnet50_transfer = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(imageSize, imageSize, 3), 
    pooling=None,
)

Resnet50_transfer.trainable = False

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
# Resnet as backbone
def get_resnet_self_supervise_model(hidden_1, hidden_2, hidden_3):
    base_model = Resnet50_transfer
    base_model.trainable = True
    inputs = Input((imageSize, imageSize, 3))
    h = base_model(inputs, training=True)
    h = GlobalAveragePooling2D()(h)

    projection_1 = Dense(hidden_1)(h)                                        
    projection_1 = Activation("relu")(projection_1)
    projection_1 = BatchNormalization(epsilon=0.001)(projection_1)
    projection_2 = Dense(hidden_2)(projection_1)
    projection_2 = Activation("relu")(projection_2)
    projection_2 = BatchNormalization(epsilon=0.001)(projection_2)
    projection_3 = Dense(hidden_3)(projection_2)
    projection_3 = BatchNormalization(epsilon=0.001)(projection_3)

    resnet_model = Model(inputs, projection_3)
    
    return resnet_model

In [ ]:
# state-of-the-arts simclr encoder trained on imagenet
saved_model_path = 'gs://simclr-checkpoints-tf2/simclrv2/pretrained/r50_1x_sk0/saved_model/'
imagenet_simclr_model = tf.saved_model.load(saved_model_path)

for image_batch in train_ds.take(1):
  simclr_logits = imagenet_simclr_model(image_batch, trainable=False)

simclr_layer_shape = simclr_logits['final_avg_pool'].shape[1]

# obtain feature maps from the state-of-the-arts simclr encoder trained on imagenet
imagenet_simclr_feature_map = np.zeros((len(peptide_morph_images_train), simclr_layer_shape))

counter = 0
for train_image_batch in train_ds:
  imagenet_simclr_feature_map[counter * training_batch_size: (counter + 1) * training_batch_size] = imagenet_simclr_model(train_image_batch, trainable=False)['final_avg_pool']
  counter += 1

## Initiate downstream classification model

In [ ]:
def get_linear_model(features):                                                                                  
    linear_model = Sequential([                                                                                  
			                              Input(shape=(features,)),
		                                Dense(4, activation="softmax")])
    return linear_model

## Performance assessment of label-efficient training of downstream classification task in main manuscript

In [ ]:
# Random seed fix
random_seed_list = np.array([42, 43, 44, 45, 46])
random_seed_for_split = np.linspace(42, 42 + 19, 20).astype(int)
training_image_size = np.array([80, 32, 16, 8, 4, 2, 1])

# list of models
models = ['barlow_tem', 'simclr_tem', 'barlow_ImageNet', 'simclr_ImageNet', 'simclr_ImageNet_sota']


earlystop_criterion = EarlyStopping(monitor='val_accuracy', patience=10, verbose=0, mode='auto', restore_best_weights=True)
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
metrics = ['accuracy']

In [ ]:
# for self-supervised models trained on tem images or ImageNet images (832 images)
for m in range(0, 4):
  linear_scores = np.zeros((len(training_image_size), len(random_seed_list), len(random_seed_for_split), 4))
  fusion_matrix = np.zeros((len(training_image_size), len(random_seed_list), len(random_seed_for_split), 4, 4))
    
  for i in range(len(random_seed_list)):
    resnet_model = get_resnet_self_supervise_model(128, 64, 1024)
    if models[m] == 'barlow_tem':
      resnet_model.load_weights('barlow_resnet_batch64_project128_64_1024_seed%i.h5' % (random_seed_list[i]))
    if models[m] == 'simclr_tem':
      resnet_model.load_weights('simclr_resnet_batch64_project128_64_1024_nocrop_seed%i.h5' % (random_seed_list[i]))
    if models[m] == 'barlow_ImageNet':
      resnet_model.load_weights('%s_batch64_project128_64_1024_seed%i.h5' % (models[m], random_seed_list[i]))
    if models[m] == 'simclr_ImageNet':
      resnet_model.load_weights('%s_batch64_project128_64_1024_nocrop_seed%i.h5' % (models[m], random_seed_list[i]))


    resnet_model.layers[1].trainable = False

    feature_extraction_model = Model(resnet_model.input, resnet_model.layers[-9].output)

    # Extract train and test features
    features = feature_extraction_model.predict(train_ds)

    for j in range(len(random_seed_for_split)):
      train_feature, test_feature, train_label, test_label = train_test_split(features, peptide_morph_train_enc, test_size=0.2, shuffle=True, stratify=peptide_morph_train_enc, random_state=random_seed_for_split[j])
      train_filename, test_filename, train_label, test_label = train_test_split(peptide_morph_images_train, peptide_morph_train_enc, test_size=0.2, shuffle=True, stratify=peptide_morph_train_enc, random_state=random_seed_for_split[j])      
      train_label_storage = pd.DataFrame(np.concatenate((train_filename.reshape(1, -1).transpose(), train_label.reshape(1, -1).transpose()), axis=-1), columns=['filename', 'label'])
      test_label_storage = pd.DataFrame(np.concatenate((test_filename.reshape(1, -1).transpose(), test_label.reshape(1, -1).transpose()), axis=-1), columns=['filename', 'label'])
    
      for n in range(len(training_image_size)):
        train_images_directory = train_label_storage.groupby('label').sample(n=training_image_size[n], random_state = 42).sample(frac=1, random_state = 42)['filename'].to_numpy().astype(str)
        train_images_label = train_label_storage.groupby('label').sample(n=training_image_size[n], random_state = 42).sample(frac=1, random_state = 42)['label'].to_numpy().astype(int)
        train_dataframe = pd.DataFrame(np.concatenate((train_images_directory.reshape(1, -1).transpose(), train_images_label.reshape(1, -1).transpose()), axis=-1), columns=['filename', 'label'])
      
        checkpoint_model_linear = ModelCheckpoint('best_classify_models_main_result/%s_%iper_class_seed%i_seed%i_linear.h5' 
                                      % (models[m], training_image_size[n], random_seed_list[i], random_seed_for_split[j]),
                                      monitor='val_accuracy', mode='auto', verbose=0, save_best_only=True, save_weights_only=True)
        train_feature_map = train_feature[train_label_storage.groupby('label').sample(training_image_size[n], random_state = 42).sample(frac=1, random_state = 42).index.to_numpy()]
        # train linear classifier model
        linear_model = get_linear_model(train_feature.shape[1])
        linear_model.compile(loss="sparse_categorical_crossentropy", metrics=metrics, optimizer=adam)
        linear_history = linear_model.fit(train_feature_map, train_images_label, validation_data=(test_feature, test_label), 
                                              batch_size=training_batch_size, epochs=300, workers=8, use_multiprocessing=True, 
                                              verbose=1, callbacks=[earlystop_criterion, checkpoint_model_linear])
        # log best classification model performance
        linear_model.load_weights('best_classify_models_main_result/%s_%iper_class_seed%i_seed%i_linear.h5' 
                                    % (models[m], training_image_size[n], random_seed_list[i], random_seed_for_split[j]))
        y_pred_linear = np.argmax(linear_model.predict(test_feature), axis=-1)

        linear_scores[n, i, j] = np.array([accuracy_score(y_pred_linear, test_label), 
                                                precision_score(y_pred_linear, test_label, average='weighted'), 
                                                recall_score(y_pred_linear, test_label, average='weighted'),
                                                f1_score(y_pred_linear, test_label, average='weighted')])
        fusion_matrix[n, i, j] = confusion_matrix(y_pred_linear, test_label)
    
          
  np.savez_compressed('classify_scores_main_result/%s_classification_result.npz' %(models[m]), scores=linear_scores, fusion_matrix=fusion_matrix)

Epoch 1/300
80/80 [==============================] - 1s 4ms/step - loss: 0.5769 - accuracy: 0.8062 - val_loss: 0.2963 - val_accuracy: 0.9000
Epoch 2/300
80/80 [==============================] - 0s 3ms/step - loss: 0.1420 - accuracy: 0.9438 - val_loss: 0.2269 - val_accuracy: 0.9125
Epoch 3/300
80/80 [==============================] - 0s 3ms/step - loss: 0.0924 - accuracy: 0.9688 - val_loss: 0.1611 - val_accuracy: 0.9375
Epoch 4/300
80/80 [==============================] - 0s 3ms/step - loss: 0.0655 - accuracy: 0.9812 - val_loss: 0.1380 - val_accuracy: 0.9500
Epoch 5/300
80/80 [==============================] - 0s 3ms/step - loss: 0.0499 - accuracy: 0.9844 - val_loss: 0.1355 - val_accuracy: 0.9500
Epoch 6/300
80/80 [==============================] - 0s 3ms/step - loss: 0.0313 - accuracy: 0.9937 - val_loss: 0.1425 - val_accuracy: 0.9375
Epoch 7/300
80/80 [==============================] - 0s 2ms/step - loss: 0.0292 - accuracy: 0.9937 - val_loss: 0.1288 - val_accuracy: 0.9500
Epoch 8/300
8

KeyboardInterrupt: ignored

In [ ]:
# for sota self-supervised encoder obtained from SimCLR official implementation

linear_scores = np.zeros((len(training_image_size), len(random_seed_for_split), 4))
fusion_matrix = np.zeros((len(training_image_size), len(random_seed_for_split), 4, 4))

for j in range(len(random_seed_for_split)):

  train_feature, test_feature, train_label, test_label = train_test_split(imagenet_simclr_feature_map, peptide_morph_train_enc, test_size=0.2, shuffle=True, stratify=peptide_morph_train_enc, random_state=random_seed_for_split[j])
  
  # reason why split is used twice: filename, feature_map and labels all need to apply train_test_split, cannot do it in one command, but since they were originally in sync
  # they can be done in two commands without issue

  train_filename, test_filename, train_label, test_label = train_test_split(peptide_morph_images_train, peptide_morph_train_enc, test_size=0.2, shuffle=True, stratify=peptide_morph_train_enc, random_state=random_seed_for_split[j])      
  train_label_storage = pd.DataFrame(np.concatenate((train_filename.reshape(1, -1).transpose(), train_label.reshape(1, -1).transpose()), axis=-1), columns=['filename', 'label'])
  test_label_storage = pd.DataFrame(np.concatenate((test_filename.reshape(1, -1).transpose(), test_label.reshape(1, -1).transpose()), axis=-1), columns=['filename', 'label'])

  for n in range(len(training_image_size)):
    train_images_directory = train_label_storage.groupby('label').sample(n=training_image_size[n], random_state = 42).sample(frac=1, random_state = 42)['filename'].to_numpy().astype(str)
    train_images_label = train_label_storage.groupby('label').sample(n=training_image_size[n], random_state = 42).sample(frac=1, random_state = 42)['label'].to_numpy().astype(int)
    train_dataframe = pd.DataFrame(np.concatenate((train_images_directory.reshape(1, -1).transpose(), train_images_label.reshape(1, -1).transpose()), axis=-1), columns=['filename', 'label'])
    
    checkpoint_model_linear = ModelCheckpoint('best_classify_models_main_result/%s_%iper_class_seed%i_linear.h5' 
                                  % (models[-1], training_image_size[n], random_seed_for_split[j]),
                                  monitor='val_accuracy', mode='auto', verbose=0, save_best_only=True, save_weights_only=True)   

    train_feature_map = train_feature[train_label_storage.groupby('label').sample(training_image_size[n], random_state = 42).sample(frac=1, random_state = 42).index.to_numpy()]
    # train linear classifier model
    linear_model = get_linear_model(train_feature.shape[1])
    linear_model.compile(loss="sparse_categorical_crossentropy", metrics=metrics, optimizer=adam)
    linear_history = linear_model.fit(train_feature_map, train_images_label, validation_data=(test_feature, test_label), 
                                          batch_size=training_batch_size, epochs=300, workers=8, use_multiprocessing=True, 
                                          verbose=1, callbacks=[earlystop_criterion, checkpoint_model_linear])
    # log best classification model performance
    linear_model.load_weights('best_classify_models_main_result/%s_%iper_class_seed%i_linear.h5' 
                                % (models[-1], training_image_size[n], random_seed_for_split[j]))
    y_pred_linear = np.argmax(linear_model.predict(test_feature), axis=-1)

    linear_scores[n, j] = np.array([accuracy_score(y_pred_linear, test_label), 
                                            precision_score(y_pred_linear, test_label, average='weighted'), 
                                            recall_score(y_pred_linear, test_label, average='weighted'),
                                            f1_score(y_pred_linear, test_label, average='weighted')])

  np.savez_compressed('classify_scores_main_result/%s_classification_result.npz' %(models[-1]), scores=linear_scores, fusion_matrix=fusion_matrix)

Epoch 1/300
80/80 [==============================] - 1s 8ms/step - loss: 1.3013 - accuracy: 0.5531 - val_loss: 1.1933 - val_accuracy: 0.7750
Epoch 2/300
80/80 [==============================] - 0s 3ms/step - loss: 1.1524 - accuracy: 0.7156 - val_loss: 1.0583 - val_accuracy: 0.6875
Epoch 3/300
80/80 [==============================] - 0s 3ms/step - loss: 1.0377 - accuracy: 0.7625 - val_loss: 0.9568 - val_accuracy: 0.7375
Epoch 4/300
80/80 [==============================] - 0s 3ms/step - loss: 0.9483 - accuracy: 0.7750 - val_loss: 0.8755 - val_accuracy: 0.8625
Epoch 5/300
80/80 [==============================] - 0s 3ms/step - loss: 0.8807 - accuracy: 0.8000 - val_loss: 0.8158 - val_accuracy: 0.8125
Epoch 6/300
80/80 [==============================] - 0s 3ms/step - loss: 0.8210 - accuracy: 0.8062 - val_loss: 0.7607 - val_accuracy: 0.8625
Epoch 7/300
80/80 [==============================] - 0s 3ms/step - loss: 0.7733 - accuracy: 0.8125 - val_loss: 0.7219 - val_accuracy: 0.8500
Epoch 8/300
8

Epoch 1/300
2/2 [==============================] - 1s 437ms/step - loss: 1.4154 - accuracy: 0.1250 - val_loss: 1.3784 - val_accuracy: 0.3625
Epoch 2/300
2/2 [==============================] - 0s 61ms/step - loss: 1.3297 - accuracy: 0.5000 - val_loss: 1.3522 - val_accuracy: 0.4500
Epoch 3/300
2/2 [==============================] - 0s 61ms/step - loss: 1.2370 - accuracy: 1.0000 - val_loss: 1.3206 - val_accuracy: 0.5625
Epoch 4/300
2/2 [==============================] - 0s 50ms/step - loss: 1.1518 - accuracy: 1.0000 - val_loss: 1.2923 - val_accuracy: 0.5625
Epoch 5/300
2/2 [==============================] - 0s 86ms/step - loss: 1.0622 - accuracy: 1.0000 - val_loss: 1.2617 - val_accuracy: 0.6000
Epoch 6/300
2/2 [==============================] - 0s 63ms/step - loss: 0.9786 - accuracy: 1.0000 - val_loss: 1.2334 - val_accuracy: 0.6375
Epoch 7/300
2/2 [==============================] - 0s 82ms/step - loss: 0.8901 - accuracy: 1.0000 - val_loss: 1.2058 - val_accuracy: 0.6250
Epoch 8/300
2/2 [==

Streaming output truncated to the last 5000 lines.
Epoch 14/300
1/1 [==============================] - 0s 62ms/step - loss: 0.5574 - accuracy: 1.0000 - val_loss: 1.2565 - val_accuracy: 0.3500
Epoch 15/300
1/1 [==============================] - 0s 66ms/step - loss: 0.5106 - accuracy: 1.0000 - val_loss: 1.2510 - val_accuracy: 0.3250
Epoch 16/300
1/1 [==============================] - 0s 91ms/step - loss: 0.4679 - accuracy: 1.0000 - val_loss: 1.2460 - val_accuracy: 0.3125
Epoch 1/300
80/80 [==============================] - 1s 8ms/step - loss: 1.0746 - accuracy: 0.6156 - val_loss: 0.8860 - val_accuracy: 0.7750
Epoch 2/300
80/80 [==============================] - 0s 4ms/step - loss: 0.7443 - accuracy: 0.8219 - val_loss: 0.7876 - val_accuracy: 0.7125
Epoch 3/300
80/80 [==============================] - 0s 3ms/step - loss: 0.6527 - accuracy: 0.8406 - val_loss: 0.7231 - val_accuracy: 0.7750
Epoch 4/300
80/80 [==============================] - 0s 3ms/step - loss: 0.5885 - accuracy: 0.8406 - va

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/300
80/80 [==============================] - 1s 8ms/step - loss: 1.0818 - accuracy: 0.6375 - val_loss: 0.8484 - val_accuracy: 0.8625
Epoch 2/300
80/80 [==============================] - 0s 3ms/step - loss: 0.7726 - accuracy: 0.8031 - val_loss: 0.7413 - val_accuracy: 0.7000
Epoch 3/300
80/80 [==============================] - 0s 3ms/step - loss: 0.6798 - accuracy: 0.8250 - val_loss: 0.6545 - val_accuracy: 0.8375
Epoch 4/300
80/80 [==============================] - 0s 3ms/step - loss: 0.6181 - accuracy: 0.8375 - val_loss: 0.6069 - val_accuracy: 0.8750
Epoch 5/300
80/80 [==============================] - 0s 3ms/step - loss: 0.5822 - accuracy: 0.8375 - val_loss: 0.5808 - val_accuracy: 0.8875
Epoch 6/300
80/80 [==============================] - 0s 3ms/step - loss: 0.5475 - accuracy: 0.8562 - val_loss: 0.5532 - val_accuracy: 0.8750
Epoch 7/300
80/80 [==============================] - 0s 3ms/step - loss: 0.5208 - accuracy: 0.8656 - val_loss: 0.5388 - val_accuracy: 0.8875
Epoch 8/300
8

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Streaming output truncated to the last 5000 lines.
80/80 [==============================] - 0s 3ms/step - loss: 0.4456 - accuracy: 0.8781 - val_loss: 0.5297 - val_accuracy: 0.8500
Epoch 12/300
80/80 [==============================] - 0s 3ms/step - loss: 0.4320 - accuracy: 0.8875 - val_loss: 0.5208 - val_accuracy: 0.8500
Epoch 13/300
80/80 [==============================] - 0s 4ms/step - loss: 0.4196 - accuracy: 0.8781 - val_loss: 0.5129 - val_accuracy: 0.8500
Epoch 14/300
80/80 [==============================] - 0s 3ms/step - loss: 0.4082 - accuracy: 0.8906 - val_loss: 0.5058 - val_accuracy: 0.8500
Epoch 15/300
80/80 [==============================] - 0s 3ms/step - loss: 0.3983 - accuracy: 0.8875 - val_loss: 0.5000 - val_accuracy: 0.8500
Epoch 1/300
32/32 [==============================] - 1s 18ms/step - loss: 1.2102 - accuracy: 0.5234 - val_loss: 1.0495 - val_accuracy: 0.7000
Epoch 2/300
32/32 [==============================] - 0s 5ms/step - loss: 0.9323 - accuracy: 0.7109 - val_loss: